# Caracterización del conjunto de datos

In [ ]:
  '''Librería OpenCV'''
import cv2
'''Librería para calculo numerico'''
import numpy as np
'''Librería para manipular rutas'''
import os
'''Para establecer variables de entorno'''
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
'''Funciones que dan soporte a la copia y remoción de archivos'''
import shutil
'''Librería para entradas que coincidan con el patrón especificado en pathname'''
import glob
'''Librería transformar las características escalando cada una de ellas a un rango determinado.'''
from sklearn.preprocessing import MinMaxScaler
'''Librería para  formato de datos jerárquico que se usa en el NILMTK como fuente datos'''
import h5py

# Carga del conjunto de datos

In [ ]:
'''Obtener la ruta de imagenes'''
images_path = '/content/drive/MyDrive/Mineria/dataset_limpio'
'''Lista con nombres de las entradas de la ruta de acceso'''
class_names = os.listdir(images_path)
'''Se ordena las entradas'''
class_names.sort()
'''Muestra las entradas'''
print(class_names)

['con_mascara', 'sin_mascara']


# Histograma

In [ ]:
''' Descriptor de característica 3: Histograma de color'''
def fd_histogram(image, mask=None):
    # convertir la imagen a espacio de color HSV
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # calcular el histograma de color
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    # normalizar el histograma
    cv2.normalize(hist, hist)
    # devolver el histograma
    return hist.flatten()

In [ ]:
'''Características de color para cada entrada'''
'''Características'''
color_features = []
'''Etiquetas'''
color_features_labels   = []

# Extracción de características

In [ ]:
'''Estado de proceso'''
print("[ESTADO] Comenzó a extraer características de color..")
'''Ciclo for para obtener el nombre de la clase'''
for class_name in class_names:
    cur_path = images_path + "/" + class_name
    cur_label = class_name
    i = 1
    
    # Ciclo for para encontrar los archivos
    for file in glob.glob(cur_path + "/*.png"):
        print("Imagen de procesamiento - {} in {}".format(i, cur_label))
        # lee la imagen de entrenamiento
        image = cv2.imread(file)

        # extraer la característica de color de la imagen
        bins = 8
        hist = fd_histogram(image)

        # agregar el vector de características y la etiqueta
        color_features.append(hist)
        color_features_labels.append(cur_label)

        # Mostrar actualización de bucle
        i += 1

Se han truncado las últimas 5000 líneas del flujo de salida.
Imagen de procesamiento - 1719 in con_mascara
Imagen de procesamiento - 1720 in con_mascara
Imagen de procesamiento - 1721 in con_mascara
Imagen de procesamiento - 1722 in con_mascara
Imagen de procesamiento - 1723 in con_mascara
Imagen de procesamiento - 1724 in con_mascara
Imagen de procesamiento - 1725 in con_mascara
Imagen de procesamiento - 1726 in con_mascara
Imagen de procesamiento - 1727 in con_mascara
Imagen de procesamiento - 1728 in con_mascara
Imagen de procesamiento - 1729 in con_mascara
Imagen de procesamiento - 1730 in con_mascara
Imagen de procesamiento - 1731 in con_mascara
Imagen de procesamiento - 1732 in con_mascara
Imagen de procesamiento - 1733 in con_mascara
Imagen de procesamiento - 1734 in con_mascara
Imagen de procesamiento - 1735 in con_mascara
Imagen de procesamiento - 1736 in con_mascara
Imagen de procesamiento - 1737 in con_mascara
Imagen de procesamiento - 1738 in con_mascara
Imagen de procesami

# Estado de caracterización

In [ ]:
'''Obtener el tamaño general del vector de características'''
print("[ESTADO] tamaño de vector de función de color {}".format(np.array(color_features).shape))

[ESTADO] tamaño de vector de función de color (6718, 512)


In [ ]:
'''Obtener el tamaño total de la etiqueta de entrenamiento'''
print("[ESTADO] etiquetas de entrenamiento {}".format(np.array(color_features_labels).shape))

[ESTADO] etiquetas de entrenamiento (6718,)


# Codificación de etiquetas

In [ ]:
'''Codificar las etiquetas de destino'''
def replace(my_list, my_dict):
    return [x if x not in my_dict else my_dict[x] for x in my_list]

'''Etiquetas'''
labels_dict = {'sin_mascara': 0, 'con_mascara': 1,}
'''Codificar etiquetas de destino con un valor entre 0 y 1'''
target = replace(color_features_labels, labels_dict)
'''Impresíon de resultado'''
print("[ESTADO] etiquetas de entrenamiento codificadas...")

[ESTADO] etiquetas de entrenamiento codificadas...


In [ ]:
'''Cracterísticas de escala en el rango (0-1)'''
scaler            = MinMaxScaler(feature_range=(0, 1))
'''Transformacion de caracteristicas'''
rescaled_features = scaler.fit_transform(color_features)
print("[ESTADO] Vector de características normalizado...")
print("[ESTADO] Etiquetas de destino: {}".format(np.array(target)))
print("[ESTADO] Forma de las etiquetas de destino: {}".format(np.array(target).shape))

[ESTADO] Vector de características normalizado...
[ESTADO] Etiquetas de destino: [1 1 1 ... 0 0 0]
[ESTADO] Forma de las etiquetas de destino: (6718,)


# Guardado de características

In [ ]:
'''Guardar el vector de características usando HDF5'''
h5f_data = h5py.File('/content/drive/MyDrive/Mineria/caracteristicasColor/color_caracteristicas.h5', 'w')
h5f_data.create_dataset('mascararillasCaracteristicas', data=np.array(rescaled_features))

h5f_label = h5py.File('/content/drive/MyDrive/Mineria/caracteristicasColor/color_caracteristicas_etiquetas.h5', 'w')
h5f_label.create_dataset('mascarillasCaracteristicas_Etiquetas', data=np.array(target))

h5f_data.close()
h5f_label.close()

print("[ESTADO] Fin de la caracterización..")

[ESTADO] Fin de la caracterización..
